# Plot number of events (division, extrusion) in time

Open an EpiCured movie, track it and inspect it for events if not done yet, and directly opens up the temporal plot of the number of events

Except for the parameters that are set-up in this notebook, it will uses the parameters saved in the preferences, loaded by default when you open the interface.

*This notebook is part of EpiCure release, see https://gitlab.pasteur.fr/gletort/epicure for more informations*

In [ ]:
import epicure.epicuring as epicure
import os
import numpy as np
import napari

In [ ]:
## path to input data, and names of the movie and EpiCured segmentation
data_dir = os.path.join("..", "data", "small")
data_img = "013_crop.tif"
data_seg = "013_crop_labels.tif"

infile = os.path.join(data_dir, data_img)
segfile = os.path.join(data_dir, "epics", data_seg)

In [ ]:
## Load the EpiCure files and movies
epic = epicure.EpiCure()
napari_viewer = napari.Viewer(show=False)
epic.viewer = napari_viewer
epic.load_movie(infile)
epic.verbose = 0  ## minimal printing of information messages
epic.go_epicure("epics", segfile)

### Do tracking if it not had been done yet

In [ ]:
if epic.tracked == 0:
    epic.tracking.do_tracking()       ## Performs tracking with the default parameters. If you have saved preferences, it will use it.
    epic.save_epicures()              ## save the results in the ouput "epics" folder(s)

### Search for extrusions with `Inspect track` option.

It uses all the default parameters except the ones redefined here. So if other inspect options were selected in the preferences, they will also be runned (eg if `flag track apparition` is selected by default, it will be runned here.

In [ ]:
get_extrusions = True  ## Whether to inspect tracks now to search for extruions. Put to False if it has already been done

if get_extrusions:
    epic.inspecting.get_extrusions.setChecked( True )   ## Be sure to search for extrusions
    epic.inspecting.threshold_disparition.setText( "300" ) ## set the threshold of cell area so that a cell disparition is considered extrusion vs potential error
    epic.inspecting.inspect_tracks( subprogress=False)  ## go, look for extrusions (and other suspects if selected in the saved preferences)

### Opens up the temporal plot

Select the events to measure (division and extrusion) by default, and opens up the temporal plot window

In [ ]:
## Choose measure events parameters
epic.outputing.output_mode.setCurrentText( "All cells" )  ## save division concerning all cells (change it to the group name to save only the one linked to a given group)
epic.outputing.events_select( "division", True )   ## output division
epic.outputing.events_select( "extrusion", True )   ## output extrusion
epic.outputing.events_select( "suspect", False )   ## dont output suspects
    
## Do the temporal plot and shows it
epic.outputing.temporal_graphs_events()